#  LangChain의 RAG 콤포넌트 - 텍스트 분할기 (Text Splitter) 

## 학습 목표
1. 텍스트 분할의 필요성과 RAG 시스템에서의 역할 이해
2. 다양한 텍스트 분할 전략(문자 기반, 재귀, 토큰 기반, 의미 기반) 비교
3. 각 분할기의 파라미터(chunk_size, chunk_overlap, separator 등) 이해 및 활용
4. 실무에서 상황에 맞는 적절한 분할 전략 선택 능력 배양

### 실습 자료

- data/transformer.pdf
- data/ai.txt

---
**추가 필기**
- 청크 전략 중요
	- 청크 사이즈에 대한 최적화가 중요하다
	- 얼만큼 최적화를 하면 좋을지를 많이 생각해 볼 문제다.
	- 큰 파일을 어떻게 LLM에 전달해서 이해시킬지를 파일을 나눈것부터가 중요하다.
- 청킹 관련 문서
	- [문서링크](https://www.anthropic.com/engineering/contextual-retrieval)
	- [청크 자동화를 시도한 문서(github)](https://github.com/Marker-Inc-Korea/AutoRAG)
	- [청킹 전략 1](https://www.pinecone.io/learn/retrieval-augmented-generation/) 
	- [청킹 전략 2](https://www.pinecone.io/learn/chunking-strategies/)
- 프로젝트 2에서 진행 할 예정 : 평가 작업 
	- A, B 테스트 처럼 진행될 예정
	- Test set이 명확해야한다.
		- 이상적인 input, output은 무엇인가?

---


# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) 문서 로드`

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf', mode="single")

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

/Users/hyewonmac/study/modu_llm_5/modu_llm5_project/001_chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF 문서 개수: 1


# 텍스트 분할(Text Splitting) 

- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항:
    1. 문서의 구조와 형식
    2. 원하는 청크 크기
    3. 문맥 보존의 중요도
    4. 처리 속도 

---
**추가 필기**
- 문서 분할 기준에 대해서 잘 생각해봐야한다.
    - 문단 단위로 할지? 어떻게 할지
    - 확인 가능 문서 : [RAG Chunk 기법 확인 가능](https://milvus.io/docs/ko/how_to_enhance_your_rag.md)
- Chunk의 크기에 따라 vector의 성격이 달라진다.
    - 작은 단위의 Chunck의 장점 : 사실을 확인을 위해 세분화가 필요한 경우
    - 큰 단위의 Chunk의 장점 : 전체 맥락에서 분석이 필요한 경우 
    - 따라서, vector의 성격에 따라서 분할을 해야한다.
- 표가 나오는 경우가 제일 문제
    - 표 형식이 아닌, 줄 + 글 형태로 읽는다.
    - 수식의 경우에도 별도의 모델이 필요하다. -> 고성능 LLM에 이미 형태로 넣는 방법이 있다.
    - 따라서 표나 수식의 경우 고성능 모델을 이용해야한다.
    - 수식 변환 모델 중 유명한 것 : [mathpix](https://mathpix.com/convert?gad_source=1&gad_campaignid=16642495173&gbraid=0AAAAAoJq_6SaLk_BazyXtroY61-W7W1Bh&gclid=CjwKCAiAkbbMBhB2EiwANbxtbcKt5hdEo5u1OxC0KyHinJcP-yBK-7P-J1CV7gV7xEks9rRVMdgOwhoC15wQAvD_BwE)
    - GeminI, Cloud의 경우에는 이미지로 변환해도 잘 인식한다.
- Chnk는 8192개의 토큰의 제한이 있더라도, ... (못 들음)
---

In [4]:
print(f'첫 번째 문서: {pdf_docs[0]}')

첫 번째 문서: page_content='Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and co

In [5]:
pdf_docs[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurre

In [6]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

첫 번째 문서의 텍스트 길이: 39643


### 1. **CharacterTextSplitter**
- 가장 기본적인 분할 방식
- 문자 수를 기준으로 텍스트를 분할
- 단순하지만 문맥을 고려하지 않는다는 단점이 있음

- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

- CharacterTextSplitter 주요 파라미터

    | 파라미터 | 기본값 | 설명 |
    |---------|--------|------|
    | `separator` | `"\n\n"` | 청크를 구분하는 구분자 |
    | `chunk_size` | `1000` | 각 청크의 최대 길이 (문자 수) |
    | `chunk_overlap` | `200` | 인접 청크 간 중복되는 문자 수 (문맥 보존) |
    | `length_function` | `len` | 텍스트 길이를 계산하는 함수 |
    | `is_separator_regex` | `False` | separator가 정규표현식인지 여부 |

In [ ]:
from langchain_text_splitters import CharacterTextSplitter 

# 텍스트 분할기 초기화 (기본 설정값 적용 )
text_splitter = CharacterTextSplitter(

    # CharacterTextSplitter의 기본 설정값
    separator = "\n\n",         # 청크 구분자: 두 개의 개행문자
    is_separator_regex = False,  # 구분자가 정규식인지 여부

    # TextSplitter의 기본 설정값
    chunk_size = 1000,          # 청크 길이
    chunk_overlap = 200,        # 청크 중첩
    length_function = len,      # 길이 함수 (문자열 길이)
    keep_separator = False,     # 구분자 유지 여부
    add_start_index = False,    # 시작 인덱스 추가 여부
    strip_whitespace = True,    # 공백 제거 여부
)

# 텍스트 분할 - split_text() 메서드 사용
texts = text_splitter.split_text(long_text)

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(texts)}')

# 첫 번째 분할된 텍스트 출력
print(f'첫 번째 분할된 텍스트: {texts[0]}')

## 위 코드의 경우 줄바꿈이 많이 없어서 분할이 1번만 된다.

분할된 텍스트 개수: 1
첫 번째 분할된 텍스트: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser ∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence a

In [8]:
# Dcoument 객체 출력 (첫 번째 분할된 텍스트로 생성)
pdf_docs[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': './data/transformer.pdf', 'total_pages': 15}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIll

In [9]:
from langchain_text_splitters import CharacterTextSplitter

# 문장 구분자를 개행문자로 설정
text_splitter = CharacterTextSplitter(
    separator = " ",        # 청크 구분자: 개행문자
    chunk_size = 1000,       # 청크 길이
    chunk_overlap = 200      # 청크 중첩
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])   # 첫 번째 문서만 분할

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(chunks)}')

# 각 청크의 텍스트 길이 출력
for i, chunk in enumerate(chunks):
    print(f'청크 {i+1}의 텍스트 길이: {len(chunk.page_content)}')

# 첫 번째 청크의 텍스트 출력
print(f'첫 번째 청크의 텍스트: {chunks[0].page_content}')

분할된 텍스트 개수: 50
청크 1의 텍스트 길이: 997
청크 2의 텍스트 길이: 995
청크 3의 텍스트 길이: 993
청크 4의 텍스트 길이: 988
청크 5의 텍스트 길이: 1000
청크 6의 텍스트 길이: 995
청크 7의 텍스트 길이: 991
청크 8의 텍스트 길이: 1000
청크 9의 텍스트 길이: 999
청크 10의 텍스트 길이: 1000
청크 11의 텍스트 길이: 987
청크 12의 텍스트 길이: 999
청크 13의 텍스트 길이: 998
청크 14의 텍스트 길이: 993
청크 15의 텍스트 길이: 998
청크 16의 텍스트 길이: 992
청크 17의 텍스트 길이: 999
청크 18의 텍스트 길이: 996
청크 19의 텍스트 길이: 1000
청크 20의 텍스트 길이: 996
청크 21의 텍스트 길이: 998
청크 22의 텍스트 길이: 991
청크 23의 텍스트 길이: 997
청크 24의 텍스트 길이: 996
청크 25의 텍스트 길이: 999
청크 26의 텍스트 길이: 987
청크 27의 텍스트 길이: 990
청크 28의 텍스트 길이: 997
청크 29의 텍스트 길이: 995
청크 30의 텍스트 길이: 998
청크 31의 텍스트 길이: 997
청크 32의 텍스트 길이: 990
청크 33의 텍스트 길이: 992
청크 34의 텍스트 길이: 997
청크 35의 텍스트 길이: 999
청크 36의 텍스트 길이: 997
청크 37의 텍스트 길이: 998
청크 38의 텍스트 길이: 993
청크 39의 텍스트 길이: 998
청크 40의 텍스트 길이: 998
청크 41의 텍스트 길이: 995
청크 42의 텍스트 길이: 999
청크 43의 텍스트 길이: 991
청크 44의 텍스트 길이: 986
청크 45의 텍스트 길이: 1000
청크 46의 텍스트 길이: 1000
청크 47의 텍스트 길이: 995
청크 48의 텍스트 길이: 996
청크 49의 텍스트 길이: 993
청크 50의 텍스트 길이: 265
첫 번째 청크의 텍스트: Provided proper attribut

### 2. **RecursiveCharacterTextSplitter**

- 재귀적으로 텍스트를 분할
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 분할
- 문맥을 더 잘 보존할 수 있음  


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,             # 청크 크기  
    chunk_overlap=200,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    # separators=["\n\n",  "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
    separators=["\n\n",  "\n", r'(?<=[.!?])\s+'],  # 강사님이 하는 방식
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("-" * 100)
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 50
각 청크의 길이: [987, 910, 975, 983, 979, 943, 930, 935, 979, 973, 954, 982, 903, 950, 957, 936, 909, 970, 948, 973, 946, 997, 982, 993, 953, 980, 987, 907, 949, 933, 979, 954, 953, 912, 991, 901, 964, 978, 964, 991, 968, 932, 954, 933, 923, 967, 947, 933, 999, 474]

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need

----------------------------------------------------------------------------------------------------
the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions

mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine transla
----------------

### 3. **정규표현식 사용**

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = CharacterTextSplitter(
    separator=r'(?<=[.!?])\s+',  # 각 Document 객체의 page_content 속성을 문장으로 분할
    chunk_size=1000,
    chunk_overlap=200,
    is_separator_regex=True,      # 구분자가 정규식인지 여부: True
    keep_separator=True,          # 구분자 유지 여부: True
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)  # 모든 문서를 분할
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200]) # 앞에서 200개
    print("-" * 100)
    print(chunk.page_content[-200:]) # 뒤에서 200개
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 50
각 청크의 길이: [997, 945, 986, 930, 996, 796, 934, 963, 880, 887, 970, 931, 712, 973, 948, 943, 840, 997, 943, 898, 938, 985, 996, 964, 920, 703, 792, 993, 969, 921, 628, 822, 944, 996, 997, 869, 854, 987, 942, 924, 967, 997, 998, 981, 917, 944, 893, 951, 976, 877]

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need

----------------------------------------------------------------------------------------------------
r and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely.

We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks 
----------------

---

### 4. **토큰 수를 기반으로 분할**

`(1) tiktoken`  
- LangChain OpenAI에서 만든 BPE Tokenizer

---
**추가 필기**
- token수로 나눈 청크가 오래 걸린다.

- Chunk 관련 질문
	- Q : 근데 토큰 단위로 하면 문맥을 유지하기 위해 오버랩을 줄 때 더 넓은 범위로 오버랩해야 문맥이 유지되는 건가요?	
	- Q : 청킹할 때 LLM 모델 이용해서 중복 문맥을 정하게 한 뒤에 청킹하는 방법도 있나요?
	- A : https://www.anthropic.com/engineering/contextual-retrieval 문서를 통해서 전달
		- hybrid로 하는 경우 성능이 더 좋다.
		- 문서 단위로 나누는게 더 좋다.
---

In [12]:
len(pdf_docs)

1

In [13]:
# 첫번째 문서 객체의 텍스트 길이
len(pdf_docs[0].page_content)

39643

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # encoding_name="cl100k_base", 
    model_name="gpt-4.1", # 이 모델이 사용하는 tokenizer을 이용하는 경우 청크 수 : 36
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])  # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(chunk.page_content[:50])
    print("-" * 50)
    print(chunk.page_content[-50:])
    print("=" * 50)
    print()

생성된 청크 수: 36
각 청크의 길이: [1146, 1374, 1376, 1503, 1425, 1217, 1357, 1307, 1229, 1182, 1333, 1147, 1204, 1423, 1323, 1253, 959, 678, 1087, 1225, 582, 918, 1368, 737, 1344, 1029, 835, 995, 933, 947, 954, 834, 993, 879, 770, 740]
Provided proper attribution is provided, Google he
--------------------------------------------------
ng more parallelizable and requiring significantly

less time to train. Our model achieves 28.4 BLEU o
--------------------------------------------------
countless long days designing various parts of and

implementing tensor2tensor, replacing our earlier 
--------------------------------------------------
 through factorization tricks [21] and conditional

computation [32], while also improving model perfo
--------------------------------------------------
istant positions [ 12]. In the Transformer this is

reduced to a constant number of operations, albeit
--------------------------------------------------
symbol representations (x1, ..., xn) to a sequence



In [25]:
import tiktoken

# tokenizer = tiktoken.get_encoding("cl100k_base")
tokenizer = tiktoken.encoding_for_model("gpt-4.1")

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (앞에서 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = [tokenizer.decode([token]) for token in tokens[:10]]
    print(token_strings)

    print("=" * 50)
    print()

280
[110436, 7937, 118839, 382, 5181, 11, 5800, 43378, 36800, 14158]
['Provided', ' proper', ' attribution', ' is', ' provided', ',', ' Google', ' hereby', ' grants', ' permission']

289
[2695, 1058, 316, 8513, 13, 5339, 2359, 136969, 220, 2029]
['less', ' time', ' to', ' train', '.', ' Our', ' model', ' achieves', ' ', '28']

286
[105849, 289, 33686, 17, 102370, 11, 39866, 1039, 11965, 3490]
['implement', 'ing', ' tensor', '2', 'tensor', ',', ' replacing', ' our', ' earlier', ' code']

289
[639, 47913, 723, 1398, 2155, 2049, 1217, 23053, 2359, 6198]
['com', 'putation', ' [', '32', '],', ' while', ' also', ' improving', ' model', ' performance']

286
[264, 77024, 316, 261, 9594, 2086, 328, 12084, 11, 84280]
['re', 'duced', ' to', ' a', ' constant', ' number', ' of', ' operations', ',', ' albeit']



In [16]:
chunk.metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-04-10T21:11:43+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2024-04-10T21:11:43+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': './data/transformer.pdf',
 'total_pages': 15}

In [17]:
print(chunk.page_content)

reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and

`(2) Hugging Face 토크나이저`  
- Hugging Face tokenizer 모델의 토큰 수를 기준으로 분할

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

tokenizer

XLMRobertaTokenizer(name_or_path='BAAI/bge-m3', vocab_size=250002, model_max_length=8192, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

---

**추가 필기**	
- 위 코드 실행 시
	- model_max_length=8192
	- `0: AddedToken("<s>", rstrip=False, lstrip ...` : 처음 시작
	- `2: AddedToken("</s>", rstrip=False, lstrip...` : 끝
	- 문서에서 알 수 있음
- 아래 코드 실행 시
	- `[0, 107687, 5, 20451, 54272, 16367, 5, 2]`
	- 0 : 시작, 2: 끝을 알 수 있다.
---

In [19]:
# 토크나이저 인코딩 - 문장을 토큰(ID)으로 변환
tokens = tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

[0, 107687, 5, 20451, 54272, 16367, 5, 2]


In [20]:
# 토큰을 출력 (토큰 ID를 실제 토큰(문자열)로 변환)
print(tokenizer.convert_ids_to_tokens(tokens)) 

['<s>', '▁안녕하세요', '.', '▁반', '갑', '습니다', '.', '</s>']


In [21]:
# 디코딩 - 토큰을 문자열로 변환
print(tokenizer.decode(tokens, skip_special_tokens=True))

안녕하세요. 반갑습니다.


In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Huggingface 토크나이저를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]]) # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = tokenizer.convert_ids_to_tokens(tokens[:10]) 
    print(token_strings)

    print("=" * 50)
    print()

생성된 청크 수: 36
각 청크의 길이: [1220, 1300, 1273, 1303, 1232, 1081, 1030, 1162, 1222, 1130, 1064, 1131, 1047, 1078, 1232, 1302, 1174, 1109, 909, 1059, 1224, 1002, 1069, 1169, 979, 1196, 1029, 944, 1027, 1001, 1045, 925, 968, 1149, 1158, 664]

299
[0, 123089, 71, 27798, 99, 179236, 83, 62952, 4, 1815]
['<s>', '▁Provide', 'd', '▁proper', '▁at', 'tribution', '▁is', '▁provided', ',', '▁Google']

295
[0, 47, 9, 191697, 153648, 66211, 4, 224588, 645, 70]
['<s>', '▁to', '-', 'German', '▁translation', '▁task', ',', '▁improving', '▁over', '▁the']

302
[0, 29479, 214, 1492, 4970, 304, 510, 4970, 4, 456]
['<s>', '▁implement', 'ing', '▁ten', 'sor', '2', 'ten', 'sor', ',', '▁re']

287
[0, 88551, 136912, 7, 23, 181135, 43315, 227066, 8305, 31461]
['<s>', '▁significant', '▁improvement', 's', '▁in', '▁computa', 'tional', '▁efficiency', '▁through', '▁factor']

282
[0, 70, 14012, 111, 41018, 7, 56065, 47, 33444, 13]
['<s>', '▁the', '▁number', '▁of', '▁operation', 's', '▁required', '▁to', '▁relat', 'e']



---

# [실습 프로젝트]

### 실습 목표
다양한 텍스트 분할 전략을 직접 구현하고 비교하면서, 각 분할기의 특성과 적합한 사용 사례를 이해합니다.

### 실습 단계
1. **문서 로드**: data/ai.txt 파일을 읽어서 랭체인 Document 객체로 변환
2. **다음 세 가지 분할기 구현**:
   - RecursiveCharacterTextSplitter (chunk_size=500, chunk_overlap=50)
   - CharacterTextSplitter (정규식 사용, 문장 단위 분할)
3. **결과 비교 분석**:
   - 각 분할기로 생성된 청크 개수
   - 첫 3개 청크의 시작 부분 비교
   - 각 분할기의 장단점 분석

### 힌트
- TextLoader를 사용하여 텍스트 파일 로드
- 각 분할기의 split_documents() 메서드 사용

In [23]:
# 여기에 코드를 작성하세요.

# 1. 문서 로드


# 2. RecursiveCharacterTextSplitter 구현


# 3. CharacterTextSplitter (정규식) 구현


# 4. 결과 비교

### chunk_size와 chunk_overlap 설정 가이드
- **chunk_size 권장 범위**: 500-1000 문자 또는 200-500 토큰
	- 200-500이 1페이지 정도
- **chunk_overlap 권장**: chunk_size의 10-20%